In [68]:
!nvidia-smi

Sat Jul 31 22:37:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [69]:
import numpy as np
import pandas as pd

In [55]:
tweets = pd.read_csv('train.csv', encoding = 'latin-1')
tweets = tweets.sample(frac=1).reset_index(drop=True)
tweets.tail()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1599994,4,2013974287,Wed Jun 03 00:05:52 PDT 2009,NO_QUERY,iledere,Well done for telling us about #BookCrossing o...
1599995,4,1835722811,Mon May 18 06:36:58 PDT 2009,NO_QUERY,JRKNaughtyNurse,@ardeming Glad to hearyou made it back to dry ...
1599996,0,2324584581,Thu Jun 25 03:31:54 PDT 2009,NO_QUERY,linaababyy,My english is getting worse . aish .. boohooho...
1599997,0,2048531661,Fri Jun 05 15:21:09 PDT 2009,NO_QUERY,CatWheatley,@SammyLou3 ohhh i see bad times my love! im h...
1599998,0,2065037863,Sun Jun 07 07:45:17 PDT 2009,NO_QUERY,hellocuppycake,@Lipe_Br grr it keeps on crashing for me


In [56]:
tweets = tweets.rename(columns={"0":"sentiment"})
tweets = tweets.rename(columns={"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D":"tweet"})
print(tweets.columns)

Index(['sentiment', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_', 'tweet'],
      dtype='object')


In [57]:
tweets = tweets.drop(axis=1, columns=['1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY', '_TheSpecialOne_'])
tweets

,sentiment,tweet
0,0,Mom's making me go to bed... have to get up ea...
1,4,Getting ready too see the movie awards
2,0,I'm working a split shift tonight Open and c...
3,0,Ouch. We've re-priced twice for worse rates to...
4,4,gaurav is making me feel hungry : stop being s...
...,...,...
1599994,4,Well done for telling us about #BookCrossing o...
1599995,4,@ardeming Glad to hearyou made it back to dry ...
1599996,0,My english is getting worse . aish .. boohooho...
1599997,0,@SammyLou3 ohhh i see bad times my love! im h...


In [59]:
tweets = tweets.head(50000)

In [60]:
for i in range(len(tweets['tweet'])):
  if tweets['sentiment'][i] == 1:
    tweets['sentiment'][i] = 0
  elif tweets['sentiment'][i] == 2:
    tweets['sentiment'][i] = 1
  elif tweets['sentiment'][i] == 4 or tweets['sentiment'][i] == 3:
    tweets['sentiment'][i] = 2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [61]:
tweets.head()

,sentiment,tweet
0,0,Mom's making me go to bed... have to get up ea...
1,2,Getting ready too see the movie awards
2,0,I'm working a split shift tonight Open and c...
3,0,Ouch. We've re-priced twice for worse rates to...
4,2,gaurav is making me feel hungry : stop being s...


In [62]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tweets['tweet'], tweets['sentiment'], test_size=0.33)

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)

In [64]:
import time
from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, y_train)
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(y_test, prediction_linear, output_dict=True)
# print('positive: ', report['pos'])
# print('negative: ', report['neg'])

Training time: 188.938856s; Prediction time: 36.376469s


In [66]:
report

{'0': {'f1-score': 0.773462783171521,
  'precision': 0.7694050315965184,
  'recall': 0.7775635618749247,
  'support': 8299},
 '2': {'f1-score': 0.7682971680764987,
  'precision': 0.7724639467521263,
  'recall': 0.7641751005974882,
  'support': 8201},
 'accuracy': 0.7709090909090909,
 'macro avg': {'f1-score': 0.7708799756240099,
  'precision': 0.7709344891743224,
  'recall': 0.7708693312362065,
  'support': 16500},
 'weighted avg': {'f1-score': 0.7708953159355041,
  'precision': 0.770925405123254,
  'recall': 0.7709090909090909,
  'support': 16500}}

In [70]:
review = """Amazon rainforest tribe protects their land"""
review_vector = vectorizer.transform([review]) # vectorizing
print(classifier_linear.predict(review_vector))

[2]


In [71]:
import pickle
pickle.dump(vectorizer, open('vectorizer.sav', 'wb'))
pickle.dump(classifier_linear, open('classifier.sav', 'wb'))